<a href="https://www.kaggle.com/code/japethj/trinit-ml-01?scriptVersionId=166222335" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Clone the Yolov5s from Github**

In [ ]:
!git clone https://github.com/rkuo2000/yolov5
%cd yolov5
import warnings
warnings.filterwarnings('ignore')

**Install library to enable to dowload from google drive***

In [ ]:
!pip install gdown
#https://drive.google.com/file/d/13fpsAzMSm9MuN1BYH6jTJnktcKiI5YmT/view?usp=sharing

**Prepare Dataset**

In [ ]:
# Download the dataset.
!wget https://learnopencv.s3.us-west-2.amazonaws.com/pothole_dataset.zip 
!wget https://drive.google.com/file/d/1U_VRTiaQGIl2MmN9LRzs8dbX6rq-MSlg/view?usp=sharing
# !gdown 1U_VRTiaQGIl2MmN9LRzs8dbX6rq-MSlg


In [ ]:
import shutil
import os

# Define the paths
input_path = '/kaggle/input/trinit'  # Path where the files are extracted
working_path = '/kaggle/working/yolov5'  # Working directory

# Specify the folder name (replace 'your_folder' with the actual folder name)
folder_name = 'India'

# Define the source and destination paths
source_path = os.path.join(input_path, folder_name)
destination_path = os.path.join(working_path, folder_name)

# Check if the destination directory exists
if os.path.exists(destination_path):
    # Remove the old folder in the working directory
    shutil.rmtree(destination_path)
    print(f"Old folder '{folder_name}' removed from the working directory.")

# Copy each subdirectory (train, test, valid) individually
for subfolder in ['train', 'test', 'valid']:
    source_subpath = os.path.join(source_path, subfolder)
    destination_subpath = os.path.join(destination_path, subfolder)

    # Copy the subdirectory to the working directory
    shutil.copytree(source_subpath, destination_subpath)
    print(f"Subfolder '{subfolder}' copied to the working directory.")

# List files in the working directory
working_files = os.listdir(working_path)
print("Files in the working directory:", working_files)



In [ ]:
!ls -1 India

In [ ]:
!ls -1 India/train

**Create YAML file***

In [ ]:
%%writefile data/roaddamage2022.yaml
train: ./India/train
val: ./India/valid
test: ./India/test

# Classes
nc: 8  # number of classes
names: ['D00','D01','D10','D11','D20','D40','D43','D44']  # class names

In [ ]:
!pip uninstall wandb -y

**Train the model**

In [ ]:
!python train.py --img 720 --batch 8 --epochs 50 --data roaddamage2022.yaml --cfg models/yolov5s.yaml

In [ ]:
!ls -1 runs/train/exp

In [ ]:
%cd /kaggle/working/yolov5
!ls -l

In [ ]:
%cd /kaggle/working/yolov5
from IPython.display import FileLink
FileLink(r'results.zip')

**Test the Model**

In [ ]:
!python detect.py --source India/test/images --img-size 720 --conf 0.4 --weights runs/train/exp4/weights/best.pt 

**Show Sample Results**

In [ ]:
from IPython.display import Image

In [ ]:
Image(filename='runs/train/exp/PR_curve.png')

In [ ]:
Image(filename='runs/train/exp/F1_curve.png')

In [ ]:
Image(filename='runs/train/exp/P_curve.png')

In [ ]:
Image(filename='runs/train/exp/confusion_matrix.png')

In [ ]:
Image(filename='runs/detect/exp/India_009534.jpg')

In [ ]:
Image(filename='runs/detect/exp/India_009852.jpg')

In [ ]:
Image(filename='runs/detect/exp/India_009190.jpg')

In [ ]:
Image(filename='runs/detect/exp/India_009537.jpg')

In [ ]:
Image(filename='runs/detect/exp/India_008583.jpg')

In [ ]:
Image(filename='runs/detect/exp/India_001632.jpg')

In [ ]:
from zipfile import ZipFile
import os

zipObj = ZipFile('results.zip', 'w')

for filename in os.listdir("/kaggle/working/yolov5/runs"):
    if filename.endswith(".png"):
        zipObj.write(filename)
zipObj.close()